# Segmentation workflow

In [ ]:
# Root of the IFT folder
home_dir = "../.."
test_dir = joinpath(home_dir, "test")

using Pkg
# Specifying an environment makes it easier to work with package versions
Pkg.activate(joinpath(home_dir, "notebooks/ift-notebooks"))

using IceFloeTracker
using Images
# Available test cases. Only showing Aqua truecolor for brevity. Each case has Aqua, Terra and has truecolor, falsecolor, and landmasks.
example_loc = joinpath(test_dir, "test_inputs/preprocessing_workflows/")

# Rearranged order just to make the figure prettier:
case_names = ["sea_of_okhostk-50km-20180412", "barents_kara_seas-100km-20250507",  "baffin_bay-200km-20060704", 
    "greenland_sea-50km-20180712", "east_siberian_sea-100km-20190612",  "bering_chukchi_seas-200km-20210314", 
    "beaufort_sea-50km-20070614", "hudson_bay-100km-20060513",  "laptev_sea-200km-20230613"]

# View the morphological residue files
file_names = [split(c, "-")[3]*".250m.aqua.morphed_residue.tiff" for c in case_names]
Images.mosaicview([RGB.(load(joinpath(example_loc, case, file)))
        for (case, file) in zip(case_names, file_names)], ncol=3)

In [ ]:
case = case_names[9]
print(case)
date = split(case, "-")[3]
satellite = "aqua"

lm_image = float64.(RGB.(load(joinpath(example_loc, case, join([date, "250m", satellite, "landmask.tiff"], ".")))))
fc_image = float64.(RGB.(load(joinpath(example_loc, case, join([date, "250m", satellite, "falsecolor.tiff"], ".")))))
tc_image = float64.(RGB.(load(joinpath(example_loc, case, join([date, "250m", satellite, "truecolor.tiff"], ".")))))
morphed_image = float64.(Gray.(load(joinpath(example_loc, case, join([date, "250m", satellite, "morphed_residue.tiff"], ".")))))
Images.mosaicview(tc_image, fc_image, lm_image, morphed_image, nrow=1)

In this notebook we examine the segmentation methods used to produce objects for tracking.

In [ ]:
# First set up the cloudmask and cloudmask
cloud_mask_settings = (
    prelim_threshold=53.0/255.,
    band_7_threshold=130.0/255.,
    band_2_threshold=169.0/255.,
    ratio_lower=0.0,
    ratio_upper=0.53
)

cloudmask = IceFloeTracker.create_cloudmask(fc_image; cloud_mask_settings...)
fc_img_cloudmasked = IceFloeTracker.apply_cloudmask(fc_image, cloudmask)
landmask = IceFloeTracker.create_landmask(lm_image)

In [ ]:
ice_masks_params = (
    band_7_threshold=5,
    band_2_threshold=230,
    band_1_threshold=240,
    band_7_threshold_relaxed=10,
    band_1_threshold_relaxed=190,
    possible_ice_threshold=75,
    k=3, # number of clusters for kmeans segmentation
    factor=255, # normalization factor to convert images to uint8
)

prelim_icemask_params = (radius=10, amount=2, factor=0.5)

In [ ]:
# Convert the morphological residue images into Array{Int64}
morphed_im_int = Array(Int64.(morphed_image .* 255));

In [ ]:
prelim_sizes = size(tc_image) .÷ 3
print(prelim_sizes)
tiles = IceFloeTracker.get_tiles(tc_image, prelim_sizes[1] + 1)

prelim_icemask, binarized_tiling = IceFloeTracker.get_ice_masks(
    n0f8.(fc_image), morphed_im_int, .!landmask.dilated, tiles, true; ice_masks_params...
)

In [ ]:
Images.mosaicview([tc_image, Gray.(prelim_icemask), Gray.(binarized_tiling)], nrow=1)

In [ ]:
@time segment_mask = IceFloeTracker.get_segment_mask(prelim_icemask, binarized_tiling);

In [ ]:
@time local_maxima_mask, L0mask = IceFloeTracker.watershed2(
    morphed_im_int, segment_mask, prelim_icemask
)

Images.mosaicview(Gray.(morphed_im_int./255), Gray.(segment_mask), Gray.(local_maxima_mask), nrow=1)

In [ ]:
structuring_elements = (
    se_disk1=collect(IceFloeTracker.MorphSE.StructuringElements.strel_diamond((3, 3))),
    se_disk2=IceFloeTracker.se_disk2(),
    se_disk4=IceFloeTracker.se_disk4(),
)

@time local_maxima_mask = IceFloeTracker.to_uint8(local_maxima_mask * 255)
prelim_icemask2 = IceFloeTracker._regularize(
    morphed_im_int,
    local_maxima_mask,
    segment_mask,
    L0mask,
    structuring_elements.se_disk1;
    prelim_icemask_params...,
);

In [ ]:
icemask, _ = IceFloeTracker.get_ice_masks(
    n0f8.(fc_image), prelim_icemask2, .!landmask.dilated, tiles, false; ice_masks_params...
)

se_erosion = structuring_elements.se_disk1
se_dilation = structuring_elements.se_disk2
final = IceFloeTracker.get_final(icemask, segment_mask, se_erosion, se_dilation);

In [ ]:
print("Top: Truecolor Image, Prelim icemask 2. Bottom: Final ice mask, Final segmentation")
Images.mosaicview(tc_image, Gray.(icemask), Gray.(prelim_icemask2./255),  Gray.(final), nrow=2)